In [1]:
import csv
import os
from datetime import datetime
import json
import psycopg2
import sys
sys.path.insert(0,'..')

from scripts.database import get_conn_pool, execute, fetch_all
from scripts.routing import route_nogo, route_standard

In [2]:
# connect to postgis database
pool = get_conn_pool({
    'host': 'localhost',
    'port': 5432,
    'user': 'postgres',
    'password': 'postgres',
    'database': 'xlab-routing'
})

In [3]:
# set start and end points

# START COORDINATES
start_lat = -1.271346
start_long = 36.751293

# END COORDINATES
end_lat = -1.190509
end_long = 36.726833

In [4]:
# set nogo layers
nogo_layer_list = ['hgis_admin3', 'hgis_bridges_tunnels']

# set nogo layer queries
nogo_layer_query_list = ["hgis_admin3.adm3_name = 'Kitisuru'", 
                         "hgis_bridges_tunnels.objectid = 810 OR hgis_bridges_tunnels.objectid = 811"]

# set nogo point layer buffer distance (in degrees) 
nogo_point_buffer = .0005  #50 meters

# set nogo polyline layer buffer distance (in degrees)
nogo_line_buffer = .0001  #10 meters


In [5]:
# get route that avoids nogo geometry
route = route_nogo(nogo_layer_list, start_long, start_lat, end_long, end_lat, nogo_layer_query_list)
wkt = route[0]
length = route[1]
wkt

NOGO QUERY:  SELECT ST_GEOMFROMEWKT(ST_COLLECT(a.shape)) FROM (SELECT ST_FORCE2D(shape) as shape FROM hgis_admin3 WHERE hgis_admin3.adm3_name = 'Kitisuru' UNION SELECT ST_FORCE2D(ST_BUFFER(shape, 0.0005)) as shape FROM hgis_bridges_tunnels WHERE hgis_bridges_tunnels.objectid = 810 OR hgis_bridges_tunnels.objectid = 811) as a
START NODE:  10339
END NODE:  57155


'MULTILINESTRING((36.7506392 -1.2680459,36.7508504 -1.2681652,36.7510103 -1.2682547,36.7511532 -1.2683405,36.7511894 -1.26837,36.7512175 -1.2684022,36.7512336 -1.2684451,36.751243 -1.2684853,36.7513087 -1.2687884,36.7513389 -1.2689452,36.751351 -1.2690149,36.7513563 -1.2690465,36.7513778 -1.2690827,36.7514288 -1.2691329,36.7515112 -1.2692006,36.751591 -1.2692623,36.7516869 -1.2693294,36.751756 -1.2693736,36.7518412 -1.2694078,36.7519565 -1.2694292,36.7522046 -1.2694614),(36.7499113 -1.2669615,36.749745 -1.2673095,36.7496712 -1.2674469,36.7496592 -1.2674871,36.7496699 -1.2675193,36.7496927 -1.2675475,36.7500622 -1.2677432,36.7506392 -1.2680459),(36.7455469 -1.2667527,36.745629 -1.2667584,36.7456492 -1.2667578),(36.7456492 -1.2667578,36.7460381 -1.2667524,36.7461361 -1.2667376,36.7461883 -1.2667265),(36.7477964 -1.2663421,36.7480819 -1.2662858,36.7481846 -1.2662751,36.748275 -1.2662744,36.7483397 -1.2662878),(36.7441661 -1.2664514,36.7444071 -1.2665145),(36.7483397 -1.2662878,36.7488136 

In [6]:
from ipyleaflet import Map, WKTLayer
import re

m = Map(center=[start_lat, end_long], zoom=12)
wlayer = WKTLayer(
    wkt_string=wkt,
    hover_style={"fillColor": "red"},
)
m.add_layer(wlayer)

m

Map(center=[-1.271346, 36.726833], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…